### 1. Options for handling quotes & Escape

id,name,remarks
1,'Ramesh, K.P','Good performer'
2,'Manoj','Needs ~'special~' attention'

### 2. Comments, Multi line, leading and trailing whitespace handling, null and nan handling

### 3. Read modes in csv

### There are 3 typical read modes and the default read mode is permissive.
##### 1. permissive — All fields are set to null and corrupted records are placed in a string column called _corrupt_record
##### 	2. dropMalformed — Drops all rows containing corrupt records.
##### 3. failFast — Fails when corrupt records are encountered.

####4. Max advanced features used...
\#This is a commented line and should be ignored
"ID","Name","Age","Salary","JoinDate","LastLogin","Notes"
1,"John Doe",28,45000.50,01-2025-25,2024-01-25 10:15:45,"New employee"
2,"Jane, Smith",32,55000.00,2023-12-30,2024-01-25 14:05:10
3,"Ravi Kumar",-1,67000.75,2023-11-05,2024-02-01 08:30:00,"Null age","addon cols"
4,"李小龍",45,88000.00,2022-05-18,2024-01-19 13:45:22,"UTF-8 Chinese name"
5,"Carlos \"The Boss\" Pérez",38,72000.30,2023-02-11,2024-01-28 09:55:05,"Contains quotes"
6,"Manoj",29,50000,2024-02-10,2024-02-10 17:25:55,"Line
break
inside notes"
7,"Anita",41,na,2023-10-08,2024-02-02 11:11:11,"Salary is NaN"
8,"Robert",34,47000.20,2023-06-22,2024-01-27 18:40:40,  "Leading and trailing spaces"   
9,"",30,39000.00,2023-09-19,2024-01-26 16:20:20,"Empty name field"
10,"#NotAComment",37,51000.10,02-2025-25,2024-02-03 12:55:30,"Starts with # but not a comment"

In [0]:
csvData =""""ID","Name","Age","Salary","JoinDate","LastLogin","Notes"
1,"John Doe",28,45000.50,01-2025-25,2024-01-25 10:15:45,"New employee"
2,"Jane, Smith",32,55000.00,2023-12-30,2024-01-25 14:05:10
3,"Ravi Kumar",-1,67000.75,2023-11-05,2024-02-01 08:30:00,"Null age","addon cols"
4,"李小龍",45,88000.00,2022-05-18,2024-01-19 13:45:22,"UTF-8 Chinese name"
5,"Carlos \"The Boss\" Pérez",38,72000.30,2023-02-11,2024-01-28 09:55:05,"Contains quotes"
6,"Manoj",29,50000,2024-02-10,2024-02-10 17:25:55,"Line
break
inside notes"
7,"Anita",41,na,2023-10-08,2024-02-02 11:11:11,"Salary is NaN"
8,"Robert",34,47000.20,2023-06-22,2024-01-27 18:40:40,  "Leading and trailing spaces"   
9,"",30,39000.00,2023-09-19,2024-01-26 16:20:20,"Empty name field"
10,"#NotAComment",37,51000.10,02-2025-25,2024-02-03 12:55:30,"Starts with # but not a comment"""

In [0]:
print(csvData)

In [0]:
%sql
create database if not exists wd36schema;
create volume if not exists wd36schema.ingestion_volume;


In [0]:
%python
dbutils.fs.mkdirs("/Volumes/workspace/wd36schema/ingestion_volume/target")

In [0]:
dbutils.fs.put("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",csvData)

####5. Reading data from other formats (Try the below usecases after completing the 3-Basic-WriteOps)

####1. Reading csv data with 3 different mode based on the correupted record

1 Fail the load if the data has corrupted<br>
2 drop the records if it is corrupted<br>
3 allow the corrupted record with null values eventhough corrupted <br>

In [0]:
# mode = FailFast
spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",header=True,sep=",",inferSchema=True,mode="FailFast").show()




In [0]:

#DropMalformed

csv_df = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",header=True,sep=",",inferSchema=True,mode="DropMalformed");
display(csv_df)




In [0]:
#Permissive
df = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/target/csvout",header=True,sep=",",inferSchema=True,mode="Permissive");
df.printSchema()
lst =df.printSchema
print(lst)






Handling the corrupted data

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# ID: string, Name: string, Age: int, Salary: string, JoinDate: string, LastLogin: timestamp, Notes: string
# 1. Define schema + the special error column
schema = StructType([
     StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("Salary", StringType(), True),
    StructField("JoinDate", StringType(), True),
    StructField("LastLogin", StringType(), True),
    StructField("Notes", StringType(), True),
    StructField("_corrupt_record", StringType(), True) # Special column
])

# 2. Read with the columnNameOfCorruptRecord option
df = (spark.read
      .format("csv")
      .option("header", "true")
      .option("mode", "PERMISSIVE")
      .option("columnNameOfCorruptRecord", "_corrupt_record")
      .schema(schema)
      .load("dbfs:///Volumes/workspace/wd36schema/ingestion_volume/target/csvout"))

# 3. View only the broken records
# display(df)
display(df.filter(df._corrupt_record.isNull()))

df.write.mode("overwrite").json("/Volumes/workspace/wd36schema/ingestion_volume/target/jsonout")
#

In [0]:
df.write.mode("overwrite").xml("/Volumes/workspace/wd36schema/ingestion_volume/target/xmlout",rowTag="cust")

####2. Reading json data

In [0]:
df_json = spark.read.json("dbfs:///Volumes/workspace/wd36schema/ingestion_volume/target/jsonout/")
df_json.printSchema();

display(df_json)

####3. Reading xml data

In [0]:
spark.read.xml("/Volumes/workspace/wd36schema/ingestion_volume/target/xmlout",rowTag="cust").show(2)

In [0]:
df.write.mode("overwrite").orc("/Volumes/workspace/wd36schema/ingestion_volume/target/orcout")

####4. Reading serialized data (orc/parquet/delta)

####5. Reading delta/hive table data